In [11]:
import json
import pandas as pd
from datetime import datetime
import os

In [12]:
input_folder = '../data/data_all_months'

In [ ]:
if False:
    # Procesar cada archivo JSON en la carpeta
    for filename in os.listdir(input_folder):
        # Listas para almacenar DataFrames de cada archivo JSON procesado
        all_publishers = []
        all_records = []
        all_releases = []
        all_compiled_releases = []
        all_parties = []
        all_contracts = []
        all_awards = []
        all_tenderers = []
        all_tenders = []
        all_items = []
        all_documents = []
        all_sources = []

        
        if filename.endswith('.json'):
            # Obtener la fecha del nombre del archivo (asumiendo el formato 'data_YYYY-MM.json')
            try:
                date_str = filename.split('_')[-1].replace('.json', '')  # Extraer parte de la fecha del nombre
                file_date = datetime.strptime(date_str, '%Y-%m')
            except ValueError:
                print(f"Formato de fecha en el nombre del archivo '{filename}' no es válido.")
                continue
            
            # Cargar el JSON
            with open(os.path.join(input_folder, filename), 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Extraer la información y crear DataFrames de pandas
            # Publisher
            publisher_data = data.get('publisher', {})
            all_publishers.append({
                'publisher_id': 1,
                'name': publisher_data.get('name'),
                'uri': publisher_data.get('uri'),
                'date_extracted': file_date.strftime('%Y-%m')  # Agregar la fecha de extracción
            })

            # Record
            for record in data.get('records', []):
                record_id = record.get('ocid')
                all_records.append({
                    'record_id': record_id,
                    'ocid': record_id,
                    'publisher_id': 1,
                    'published_date': record.get('publishedDate'),
                    'license': data.get('license'),
                    'publication_policy': data.get('publicationPolicy'),
                    'version': data.get('version'),
                    'date_extracted': file_date.strftime('%Y-%m')
                })

                # Release
                for release in record.get('releases', []):
                    all_releases.append({
                        'release_id': release.get('url'),
                        'record_id': record_id,
                        'url': release.get('url'),
                        'release_date': release.get('date'),
                        'date_extracted': file_date.strftime('%Y-%m')
                    })

                # Compiled Release
                compiled_release = record.get('compiledRelease', {})
                all_compiled_releases.append({
                    'compiled_release_id': compiled_release.get('ocid'),
                    'record_id': record_id,
                    'compiled_date': compiled_release.get('date'),
                    'published_date': compiled_release.get('publishedDate'),
                    'language': compiled_release.get('language'),
                    'initiation_type': compiled_release.get('initiationType'),
                    'date_extracted': file_date.strftime('%Y-%m')
                })

                # Parties
                for party in compiled_release.get('parties', []):
                    all_parties.append({
                        'party_id': party.get('id'),
                        'name': party.get('name'),
                        'identifier_scheme': party.get('identifier', {}).get('scheme'),
                        'identifier_id': party.get('identifier', {}).get('id'),
                        'address_street': party.get('address', {}).get('streetAddress'),
                        'address_locality': party.get('address', {}).get('locality'),
                        'address_region': party.get('address', {}).get('region'),
                        'address_country': party.get('address', {}).get('countryName'),
                        'contact_point_name': party.get('contactPoint', {}).get('name'),
                        'contact_point_email': party.get('contactPoint', {}).get('email'),
                        'contact_point_telephone': party.get('contactPoint', {}).get('telephone'),
                        'contact_point_fax': party.get('contactPoint', {}).get('faxNumber'),
                        'roles': ', '.join(party.get('roles', [])),
                        'details_level': party.get('details', {}).get('level', {}).get('description'),
                        'details_entity_type': party.get('details', {}).get('entityType', {}).get('description'),
                        'details_type': party.get('details', {}).get('type', {}).get('description'),
                        'date_extracted': file_date.strftime('%Y-%m')
                    })

                # Tender
                tender = compiled_release.get('tender', {})
                all_tenders.append({
                    'tender_id': tender.get('id'),
                    'title': tender.get('title'),
                    'status': tender.get('status'),
                    'status_details': tender.get('statusDetails'),
                    'date_published': tender.get('datePublished'),
                    'procurement_method': tender.get('procurementMethod'),
                    'procurement_method_details': tender.get('procurementMethodDetails'),
                    'main_procurement_category': tender.get('mainProcurementCategory'),
                    'submission_method': ', '.join(tender.get('submissionMethod', [])),
                    'tender_period_start_date': tender.get('tenderPeriod', {}).get('startDate'),
                    'tender_period_end_date': tender.get('tenderPeriod', {}).get('endDate'),
                    'number_of_tenderers': tender.get('numberOfTenderers'),
                    'date_extracted': file_date.strftime('%Y-%m')
                })

                # Items
                for item in tender.get('items', []):
                    all_items.append({
                        'item_id': item.get('id'),
                        'tender_id': tender.get('id'),
                        'description': item.get('description'),
                        'quantity': item.get('quantity'),
                        'unit_name': item.get('unit', {}).get('name'),
                        'item_id_external': item.get('id'),
                        'date_extracted': file_date.strftime('%Y-%m')
                    })

                # Documents
                for document in tender.get('documents', []):
                    all_documents.append({
                        'document_id': document.get('id'),
                        'document_type': document.get('documentType'),
                        'title': document.get('title'),
                        'url': document.get('url'),
                        'format': document.get('format'),
                        'language': document.get('language'),
                        'date_published': document.get('datePublished'),
                        'document_type_details': document.get('documentTypeDetails'),
                        'tender_id': tender.get('id'),
                        'date_extracted': file_date.strftime('%Y-%m')
                    })

                # Contracts and Awards
                for award in compiled_release.get('awards', []):
                    all_awards.append({
                        'award_id': award.get('id'),
                        'title': award.get('title'),
                        'status': award.get('status'),
                        'status_details': award.get('statusDetails'),
                        'date_awarded': award.get('date'),
                        'award_value_amount': award.get('value', {}).get('amount'),
                        'award_value_currency': award.get('value', {}).get('currency'),
                        'date_extracted': file_date.strftime('%Y-%m')
                    })
                    for supplier in award.get('suppliers', []):
                        all_tenderers.append({
                            'tenderer_id': supplier.get('id'),
                            'party_id': supplier.get('id'),
                            'tender_id': tender.get('id'),
                            'date_extracted': file_date.strftime('%Y-%m')
                        })

                for contract in compiled_release.get('contracts', []):
                    all_contracts.append({
                        'contract_id': contract.get('id'),
                        'award_id': contract.get('awardID'),
                        'title': contract.get('title'),
                        'contract_number': contract.get('contractNumber'),
                        'status': contract.get('status'),
                        'status_details': contract.get('statusDetails'),
                        'date_signed': contract.get('dateSigned'),
                        'start_date': contract.get('period', {}).get('startDate'),
                        'end_date': contract.get('period', {}).get('endDate'),
                        'contract_value_amount': contract.get('value', {}).get('amount'),
                        'contract_value_currency': contract.get('value', {}).get('currency'),
                        'date_extracted': file_date.strftime('%Y-%m')
                    })
        output_file = f'output_{file_date.year}-{file_date.month}.xlsx'

        # Crear DataFrames de pandas y guardarlos en el archivo Excel
        with pd.ExcelWriter(output_file) as writer:
            pd.DataFrame(all_publishers).to_excel(writer, sheet_name='Publishers', index=False)
            pd.DataFrame(all_records).to_excel(writer, sheet_name='Records', index=False)
            pd.DataFrame(all_releases).to_excel(writer, sheet_name='Releases', index=False)
            pd.DataFrame(all_compiled_releases).to_excel(writer, sheet_name='CompiledReleases', index=False)
            pd.DataFrame(all_parties).to_excel(writer, sheet_name='Parties', index=False)
            pd.DataFrame(all_contracts).to_excel(writer, sheet_name='Contracts', index=False)
            pd.DataFrame(all_awards).to_excel(writer, sheet_name='Awards', index=False)
            pd.DataFrame(all_tenderers).to_excel(writer, sheet_name='Tenderers', index=False)
            pd.DataFrame(all_tenders).to_excel(writer, sheet_name='Tenders', index=False)
            pd.DataFrame(all_items).to_excel(writer, sheet_name='Items', index=False)
            pd.DataFrame(all_documents).to_excel(writer, sheet_name='Documents', index=False)
            pd.DataFrame(all_sources).to_excel(writer, sheet_name='Sources', index=False)


In [ ]:

'''
# Cargar el JSON desde un archivo o directamente como un diccionario
with open('../data/data_all_months/file_2024-10.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extraer la información de las tablas y crear DataFrames
publishers = []
records = []
releases = []
compiled_releases = []
parties = []
contracts = []
awards = []
tenderers = []
tenders = []
items = []
documents = []
sources = []

# Extraer los datos de la estructura JSON y llenar los DataFrames
# Publisher
publisher_data = data.get('publisher', {})
publishers.append({
    'publisher_id': 1,  # Asignar un ID único (puedes usar un contador o generador si es necesario)
    'name': publisher_data.get('name'),
    'uri': publisher_data.get('uri')
})

# Record
for record in data.get('records', []):
    record_id = record.get('ocid')
    records.append({
        'record_id': record_id,
        'ocid': record_id,
        'publisher_id': 1,  # Asumimos que solo hay un publisher por ahora
        'published_date': record.get('publishedDate'),
        'license': data.get('license'),
        'publication_policy': data.get('publicationPolicy'),
        'version': data.get('version')
    })

    # Release
    for release in record.get('releases', []):
        releases.append({
            'release_id': release.get('url'),  # Usamos la URL como identificador único (puedes usar otro campo)
            'record_id': record_id,
            'url': release.get('url'),
            'release_date': release.get('date')
        })

    # Compiled Release
    compiled_release = record.get('compiledRelease', {})
    compiled_releases.append({
        'compiled_release_id': compiled_release.get('ocid'),  # Puede usar otro identificador si es necesario
        'record_id': record_id,
        'compiled_date': compiled_release.get('date'),
        'published_date': compiled_release.get('publishedDate'),
        'language': compiled_release.get('language'),
        'initiation_type': compiled_release.get('initiationType')
    })

    # Parties
    for party in compiled_release.get('parties', []):
        parties.append({
            'party_id': party.get('id'),
            'name': party.get('name'),
            'identifier_scheme': party.get('identifier', {}).get('scheme'),
            'identifier_id': party.get('identifier', {}).get('id'),
            'address_street': party.get('address', {}).get('streetAddress'),
            'address_locality': party.get('address', {}).get('locality'),
            'address_region': party.get('address', {}).get('region'),
            'address_country': party.get('address', {}).get('countryName'),
            'contact_point_name': party.get('contactPoint', {}).get('name'),
            'contact_point_email': party.get('contactPoint', {}).get('email'),
            'contact_point_telephone': party.get('contactPoint', {}).get('telephone'),
            'contact_point_fax': party.get('contactPoint', {}).get('faxNumber'),
            'roles': ', '.join(party.get('roles', [])),
            'details_level': party.get('details', {}).get('level', {}).get('description'),
            'details_entity_type': party.get('details', {}).get('entityType', {}).get('description'),
            'details_type': party.get('details', {}).get('type', {}).get('description')
        })

    # Tender
    tender = compiled_release.get('tender', {})
    tenders.append({
        'tender_id': tender.get('id'),
        'title': tender.get('title'),
        'status': tender.get('status'),
        'status_details': tender.get('statusDetails'),
        'date_published': tender.get('datePublished'),
        'procurement_method': tender.get('procurementMethod'),
        'procurement_method_details': tender.get('procurementMethodDetails'),
        'main_procurement_category': tender.get('mainProcurementCategory'),
        'submission_method': ', '.join(tender.get('submissionMethod', [])),
        'tender_period_start_date': tender.get('tenderPeriod', {}).get('startDate'),
        'tender_period_end_date': tender.get('tenderPeriod', {}).get('endDate'),
        'number_of_tenderers': tender.get('numberOfTenderers')
    })

    # Items
    for item in tender.get('items', []):
        items.append({
            'item_id': item.get('id'),
            'tender_id': tender.get('id'),
            'description': item.get('description'),
            'quantity': item.get('quantity'),
            'unit_name': item.get('unit', {}).get('name'),
            'item_id_external': item.get('id')
        })

    # Documents
    for document in tender.get('documents', []):
        documents.append({
            'document_id': document.get('id'),
            'document_type': document.get('documentType'),
            'title': document.get('title'),
            'url': document.get('url'),
            'format': document.get('format'),
            'language': document.get('language'),
            'date_published': document.get('datePublished'),
            'document_type_details': document.get('documentTypeDetails'),
            'tender_id': tender.get('id'),
        })

    # Contracts and Awards
    for award in compiled_release.get('awards', []):
        awards.append({
            'award_id': award.get('id'),
            'title': award.get('title'),
            'status': award.get('status'),
            'status_details': award.get('statusDetails'),
            'date_awarded': award.get('date'),
            'award_value_amount': award.get('value', {}).get('amount'),
            'award_value_currency': award.get('value', {}).get('currency')
        })
        for supplier in award.get('suppliers', []):
            tenderers.append({
                'tenderer_id': supplier.get('id'),
                'party_id': supplier.get('id'),
                'tender_id': tender.get('id')
            })

    for contract in compiled_release.get('contracts', []):
        contracts.append({
            'contract_id': contract.get('id'),
            'award_id': contract.get('awardID'),
            'title': contract.get('title'),
            'contract_number': contract.get('contractNumber'),
            'status': contract.get('status'),
            'status_details': contract.get('statusDetails'),
            'date_signed': contract.get('dateSigned'),
            'start_date': contract.get('period', {}).get('startDate'),
            'end_date': contract.get('period', {}).get('endDate'),
            'contract_value_amount': contract.get('value', {}).get('amount'),
            'contract_value_currency': contract.get('value', {}).get('currency')
        })

# Crear DataFrames de pandas
df_publishers = pd.DataFrame(publishers)
df_records = pd.DataFrame(records)
df_releases = pd.DataFrame(releases)
df_compiled_releases = pd.DataFrame(compiled_releases)
df_parties = pd.DataFrame(parties)
df_contracts = pd.DataFrame(contracts)
df_awards = pd.DataFrame(awards)
df_tenderers = pd.DataFrame(tenderers)
df_tenders = pd.DataFrame(tenders)
df_items = pd.DataFrame(items)
df_documents = pd.DataFrame(documents)
df_sources = pd.DataFrame(sources)

# Guardar en un archivo Excel
with pd.ExcelWriter('output.xlsx') as writer:
    df_publishers.to_excel(writer, sheet_name='Publishers', index=False)
    df_records.to_excel(writer, sheet_name='Records', index=False)
    df_releases.to_excel(writer, sheet_name='Releases', index=False)
    df_compiled_releases.to_excel(writer, sheet_name='CompiledReleases', index=False)
    df_parties.to_excel(writer, sheet_name='Parties', index=False)
    df_contracts.to_excel(writer, sheet_name='Contracts', index=False)
    df_awards.to_excel(writer, sheet_name='Awards', index=False)
    df_tenderers.to_excel(writer, sheet_name='Tenderers', index=False)
    df_tenders.to_excel(writer, sheet_name='Tenders', index=False)
    df_items.to_excel(writer, sheet_name='Items', index=False)
    df_documents.to_excel(writer, sheet_name='Documents', index=False)
    df_sources.to_excel(writer, sheet_name='Sources', index=False)

print("El archivo Excel ha sido generado exitosamente.")
'''

El archivo Excel ha sido generado exitosamente.
